In [12]:
import pandas as pd
from pathlib import Path
import zipfile

In [28]:
# path to file
path_to_file = "../legiscan-bulk-csv/2025-2026"

save_path = "../legiscan-bulk-csv/concatenated-2025-2026/all-bills.csv"

In [29]:
INPUT_PATH = Path(path_to_file)
SAVE_PATH = Path(save_path)

# unzip (if necessary)
if INPUT_PATH.suffix == ".zip":
    if not INPUT_PATH.exists():
        raise FileNotFoundError(f"{INPUT_PATH} not found")

    root_dir = INPUT_PATH.with_suffix("")  # strip .zip

    if not root_dir.exists():
        print(f"Unzipping {INPUT_PATH}...")
        root_dir.mkdir(parents=True, exist_ok=True)
        
        with zipfile.ZipFile(INPUT_PATH, "r") as zip_ref:
            zip_ref.extractall(root_dir)
        print("Unzip complete.")
    else:
        print(f"{root_dir} already exists, skipping unzip.")
else:
    if not INPUT_PATH.exists():
        raise FileNotFoundError(f"{INPUT_PATH} not found")

    root_dir = INPUT_PATH
    print(f"Existing directory found: {root_dir}")

../legiscan-bulk-csv/2025-2026 already exists, skipping unzip.


In [36]:

from pathlib import Path
import pandas as pd

# SAVE_PATH = Path("...")  # make sure you define this
# root_dir = Path("...")   # your extracted folder root

# Collect ALL bills.csv files under root_dir
bills_paths = list(root_dir.rglob("CSV/bills.csv"))

all_bills = []

if not bills_paths:
    print("No bills.csv files found")
else:
    for bills_path in bills_paths:
        try:
            # Pull state from one level higher in the directory structure:
            # .../<STATE>/<session>/CSV/bills.csv
            state = bills_path.parents[2].name

            df = pd.read_csv(bills_path)
            df["state"] = state
            all_bills.append(df)
        except Exception as e:
            print(f"Error reading {bills_path}: {e}")

# --- Write output ---
if all_bills:
    combined = pd.concat(all_bills, ignore_index=True)

    before = len(combined)
    combined = combined.drop_duplicates()
    after = len(combined)

    SAVE_PATH.parent.mkdir(parents=True, exist_ok=True)
    combined.to_csv(SAVE_PATH, index=False)

    print(
        f"Written combined bills.csv to {SAVE_PATH} "
        f"({before} → {after} rows, {before - after} duplicates removed)"
    )
else:
    print("No bills.csv files found")


Written combined bills.csv to ../legiscan-bulk-csv/concatenated-2025-2026/all-bills.csv (142696 → 142696 rows, 0 duplicates removed)
